<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Calculate the Fosberg Fire Weather Index for Each Model and Timestep</h1>
By: Kayla Besong, PhD
    <br>
Last Edited: 11/29/23
<br>
<br>    
<br>
Takes models/variables downloaded and calculates the Fosberg Fire Weather Index. This notebook leverages already generated database files.
<div style="clear:both"></div>
</div>

<hr style="height:2px;">

## Import needed libraries, etc.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from dask.distributed import Client, LocalCluster
import dask.array as da
import os
import glob
from metpy.units import units
import math

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None

## OPTIONAL: Establish a dask client. This is a lot of data.

In [ ]:
Cluster = LocalCluster(n_workers = 8, threads_per_worker=4, memory_limit='30GB',  processes=True)
#Cluster = LocalCluster()

In [ ]:
client = Client(Cluster)
client

### The integral notebook of functions to run

In [ ]:
%run File_concat_mod_functions.ipynb

## The Fosberg function, variables, models, etc. 

In [ ]:
model_options = ['CONUS404', 'ERA5', 'HRRR', 'NAM', 'NARR', 'NCEP', 'UFS_S2S']
variable_options =  ['PBL', 'CAPE', 'SOILM', 'WIND_COMP', 'PRECIP', 'TEMP', 'RH', 'WINDSPEED', 'WINDDIR', 'HDWI', 'VPD']

In [ ]:
output_dir = 'database_files'

In [ ]:
# RH_not_in_percent = ['CONUS404', 'ERA5']
# RH_in_percent = ['NARR', 'NCEP', 'HRRR', 'UFS_S2S']

In [1]:
def get_m(h, t):

    ''' This function calculates the equilibrium moisture content (m) for the Fosberg Index given conditionals.

    Inputs:

    h: (float) relative humidity value 
    t: (float) temperature value

    Outputs:

    m: (float or nan) 

    '''
    
    if h < 10:
    
        m =0.03229 + 0.281073*h - 0.000578*h*t
    
    elif h >= 10 and h < 50:
    
        m = 2.22749 + 0.160107*h - 0.01478*t
    
    elif h >= 50:
    
        m = 21.0606 + 0.005565*h**2 - 0.00035*h*t - 0.483199*h

    else:

        m = np.nan

    return m


In [ ]:
def time_isect(tempt, rht, ws, model):

    ''' Function designed to handle problematic datasets where the data is not complete for all variables. This can be skipped if your data is already cleaned. 

    Inputs:

    tempt: (xarray dataset) input dataset containing temperature values 
    rht: (xarray dataset) input dataset containing relative humidity values in percent
    ws: (xarray dataset) input dataset containing windspeed values 
    model: (str) model name

    Outputs:

    tempt: (xarray dataset) input dataset containing temperature values with same times as rht and ws
    rht: (xarray dataset) input dataset containing relative humidity values in percent with same times as tempt and ws
    ws: (xarray dataset) input dataset containing windspeed values with same times as rht and tempt

    '''
    

    if model == 'CONUS404':                                                                                 # Check if the model is 'CONUS404'
        
        t_times = tempt.Time.values                                                                         # Get time values from temperature dataset
        r_times = rht.Time.values                                                                           # Get time values from relative humidity dataset
        w_times = ws.Time.values                                                                            # Get time values from wind speed dataset
        
        sel_times = np.intersect1d(np.intersect1d(t_times, r_times), w_times)                               # Find common time values across all datasets
    
        tempt = tempt.sel(Time = sel_times)                                                                 # Select temperature data for common times
        rht = rht.sel(Time = sel_times)                                                                     # Select relative humidity data for common times
        ws = ws.sel(Time = sel_times)                                                                       # Select wind speed data for common times

    elif model == 'NAM':                                                                                    # Check if the model is 'NAM'
        
        r_times = rht.time.values                                                                           # Get time values from relative humidity dataset
        
        matching_indices_1 = [i for i, t in enumerate(tempt.time.values) if t in r_times]                   # Find indices in temperature data that match relative humidity times

        tempt = tempt.isel(time=matching_indices_1)                                                         # Select temperature data for matching indices
        t_times = tempt.time.values                                                                         # Get updated time values from temperature dataset
        
        matching_indices_2 = [i for i, t in enumerate(ws.time.values) if t in t_times]                      # Find indices in wind speed data that match updated temperature times

        ws = ws.isel(time = matching_indices_2)                                                             # Select wind speed data for matching indices

    else:                                                                                                   # For any other model
        
        t_times = tempt.time.values                                                                         # Get time values from temperature dataset
        r_times = rht.time.values                                                                           # Get time values from relative humidity dataset
        w_times = ws.time.values                                                                            # Get time values from wind speed dataset
        
        sel_times = np.intersect1d(np.intersect1d(t_times, r_times), w_times)                               # Find common time values across all datasets
    
        tempt = tempt.sel(time = sel_times)                                                                 # Select temperature data for common times
        rht = rht.sel(time = sel_times)                                                                     # Select relative humidity data for common times
        ws = ws.sel(time = sel_times)                                                                       # Select wind speed data for common times

    return tempt, rht, ws   

In [ ]:
def ffwi(model, output_dir):


    ''' Function to compute the Fosberg Fire Weather Index given the input model and directory where data lives. 

    Inputs:
    
    model: (str) model name, used as file path 
    output_dir: (str) the directory that contains the model data 

    Outputs:
    
    Nothing, files are saved to pointed directory.  

    '''
    
    model_list = []                                                                                                     # Initialize an empty list to store file lists for each variable
                      
    parent_dir = f'{output_dir}/{model}'                                                                                # Define the parent directory path for the model
                      
    variable_options = ['TEMP', 'RH', 'WINDSPEED']                                                                      # List of variable options to process
    model_vars = []                                                                                                     # Initialize an empty list to store processed variable names
    get_m_vec = np.vectorize(get_m)                                                                                     # Vectorize the get_m function for efficient array operations
                      
    for v in variable_options:                                                                                          # Iterate over each variable option
        v = get_model_var_database(model, v)                                                                            # Retrieve the standardized variable name for the model
        model_vars.append(v)                                                                                            # Append the standardized variable name to model_vars list
        model_list.append(sorted(glob.glob(os.path.join(parent_dir, f'{v}_{get_filename(model)}_Abs_*.nc'))))           # Append sorted list of file paths for each variable
                      
    if len(np.unique([len(i) for i in model_list])) > 1:                                                                # Check if the number of files for each variable is the same
        print('the number of years for each variable are not the same')                                                 # Print a message if the number of files is not the same
                      
    else:                  
        ct = 0                                                                                                          # Initialize a counter
        for temp, rh in zip(model_list[0], model_list[1]):                                                              # Iterate over pairs of files for temperature and relative humidity
            if int(temp[-7:-3]) != int(rh[-7:-3]):                                                                      # Check if the years in the file names are aligned
                print('the years for each variable are not aligned, rh')                                                # Print a message if the years are not aligned for temperature and relative humidity
            else:                  
                ws = model_list[2][ct]                                                                                  # Retrieve the corresponding wind speed file
                      
                if int(temp[-7:-3]) != int(ws[-7:-3]):                                                                  # Check if the years in the file names are aligned for wind speed
                    print('the years for each variable are not aligned, ws')                                            # Print a message if the years are not aligned for wind speed
                else:                  
                    print(temp, rh)                                                                                     # Print the file names for temperature and relative humidity
                      
                    if model == 'NAM':                                                                                  # Special handling for NAM model that would not cooperate with DASK 
                        tempt = xr.open_dataset(temp)                                                                   # Open the dataset for temperature
                        rht = xr.open_dataset(rh)                                                                       # Open the dataset for relative humidity
                        ws = xr.open_dataset(ws)                                                                        # Open the dataset for wind speed
                    else:                 
                        tempt = xr.open_dataset(temp).chunk(get_chunk_database(model))                                  # Open and chunk the dataset for temperature
                        rht = xr.open_dataset(rh).chunk(get_chunk_database(model))                                      # Open and chunk the dataset for relative humidity
                        ws = xr.open_dataset(ws).chunk(get_chunk_database(model))                                       # Open and chunk the dataset for wind speed
                    
                    if model == 'ERA5' or model == 'CONUS404' or model == 'HRRR':                                       # Adjust relative humidity for specific models
                        rht = rht*100                                                                                   # Convert relative humidity to percentage
                    
                    tempt = (tempt - 273.15)*(9./5.)+32                                                                 # Convert temperature from Kelvin to Fahrenheit
                    
                    tempt, rht, ws = time_isect(tempt, rht, ws, model)                                                  # Intersect the time dimensions of the datasets
                    
                    m = xr.apply_ufunc(get_m_vec, rht[model_vars[1]], tempt[model_vars[0]], dask='parallelized')        # Broadcast the get_m function across the dataset, cool trick 
                    n = 1-2*(m/30)+1.5*(m/30)**2-0.5*(m/30)**3                                                          # Calculate n based on m
                    ffwi = (n*((1+ws[model_vars[2]]**2)**0.5)/0.3002).to_dataset(name='ffwi')                           # Calculate FFWI and create a new dataset
                    
                    resampler_regular_vars('ffwi', ffwi, output_dir, model)                                             # Resample and save the FFWI dataset
                    
                    ct += 1                                                                                             # Increment the counter


### Run the function by model. I split it up so I can restart the kernel in between but can be worked into a for loop.

In [ ]:
%%time

ffwi('NARR', output_dir)

In [ ]:
%%time

ffwi('ERA5', output_dir)

In [ ]:
%%time

ffwi('HRRR', output_dir)

In [ ]:
%%time

ffwi('CONUS404', output_dir)

In [ ]:
%%time

ffwi('NAM', output_dir)